In [2]:
import numpy as np
import math
import matplotlib.pyplot as plt
import copy

In [3]:
# "MAX" = -1
# "MIN" = 1
class State:
    def __init__(self, current_player="MAX"):
        self.game_board = self.initial_game_board()
        self.current_player = current_player
        
    def initial_game_board(self):
        initial_state = np.zeros(shape=(4,4), dtype=np.int32)
        initial_state[0,0] = -1
        initial_state[1,1] = -1
        initial_state[2,2] = -1
        initial_state[3,3] = -1
        initial_state[0,3] = 1
        initial_state[1,2] = 1
        initial_state[2,1] = 1
        initial_state[3,0] = 1
        return initial_state
    
    def get_value(self):
        if self.current_player == "MAX":
            return -1
        elif self.current_player == "MIN":
            return 1
    
    def set_value(self, row, col, value):
        self.game_board[row, col] = value
    
    def is_player_disk(self, row, col, value):
        return self.game_board[row, col] == value and self.game_board[row, col] != 0
    
    def is_free_space(self, row, col):
        return self.game_board[row, col] == 0
    
    def change_player(self):
        if self.current_player == "MAX":
            self.current_player = "MIN"
        else:
            self.current_player = "MAX"

In [4]:
max_limit = 3
min_limit = 0

def out_of_boundaries(pos):
    return pos < min_limit or pos > max_limit

def not_out_of_limits(row, column):
    return not out_of_boundaries(row) and not out_of_boundaries(column)

def down_function(row, column):
    return row+1, column

def up_function(row, column):
    return row-1, column

def left_function(row, column):
    return row, column-1

def right_function(row, column):
    return row, column+1

def up_left_function(row, column):
    return row-1, column-1

def up_right_function(row, column):
    return row-1, column+1

def down_left_function(row, column):
    return row+1, column-1

def down_right_function(row, column):
    return row+1, column+1

def move_function(state, row, col, move_func):
    value = state.get_value()
    if not_out_of_limits(row, col) and state.is_player_disk(row, col, value):
            new_row, new_col = move_func(row, col)
            if not_out_of_limits(new_row, new_col) and state.is_free_space(new_row, new_col):
                state.set_value(new_row, new_col, value)
                state.set_value(row, col, 0)
                state.change_player()
    return state

In [5]:
#test_input = input("Escoge con que quieres iniciar: negras o blancas, ejemplo: (NEGRO, BLANCO)")
test_input = "negro"
if test_input.upper() == "NEGRO":
    test_input = "MAX"
if test_input.upper() == "BLANCO":
    test_input = "MIN"

state_test = State()
print(f"INITIAL PLAYER: {state_test.current_player} with value {state_test.get_value()}")
print(state_test.game_board)
print("-------------")
print(f"PLAYER: {state_test.current_player} with value {state_test.get_value()} moving [0, 0] to right")
state_test = move_function(state_test, 0, 0, right_function)
print(state_test.game_board)
print("-------------")
print(f"PLAYER: {state_test.current_player} with value {state_test.get_value()} moving [1, 2] to up")
state_test = move_function(state_test, 1, 2, up_function)
print(state_test.game_board)
print("-------------")
print(f"PLAYER: {state_test.current_player} with value {state_test.get_value()} moving [1, 2] to up_left")
state_test = move_function(state_test, 1, 1, up_left_function)
print(state_test.game_board)
print("-------------")
#print(move_function(a, 1, 0, 0, down_function).game_board)
#print("-------------")
#print(move_function(a, -1, 2, 1, down_right_function).game_board)

INITIAL PLAYER: MAX with value -1
[[-1  0  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
-------------
PLAYER: MAX with value -1 moving [0, 0] to right
[[ 0 -1  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
-------------
PLAYER: MIN with value 1 moving [1, 2] to up
[[ 0 -1  1  1]
 [ 0 -1  0  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
-------------
PLAYER: MAX with value -1 moving [1, 2] to up_left
[[-1 -1  1  1]
 [ 0  0  0  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
-------------


In [6]:
def check_victory(state):
    game_board = state.game_board
    # Check horizontal columns
    for row in range(4):
        if abs(np.sum(game_board[row, :])) == 4:
            return True
    # Check vertical columns
    for col in range(4):
        if abs(np.sum(game_board[:, col])) == 4:
            return True
    # Check corner positions
    if game_board[0, 0] == game_board[0, 3] == game_board[3, 0] == game_board[3, 3] != 0:
        return True
    # Check square 2x2
    for i in range(4):
        for j in range(4):
            submatrix = game_board[i:i+2, j:j+2]
            if abs(np.sum(submatrix)) == 4:
                return True
    return False

In [7]:
victory_state = State()
victory_state.game_board = np.array([[0, 0, 0, 0],
                                     [1, 0, 1, 1],
                                     [0, 0, -1, -1],
                                     [0, 1, -1, -1]])
print(victory_state.game_board)
print(check_victory(victory_state))

[[ 0  0  0  0]
 [ 1  0  1  1]
 [ 0  0 -1 -1]
 [ 0  1 -1 -1]]
True


In [8]:
def possible_move_function(state, row, col, move_func):
    copy_state = copy.deepcopy(state)
    value = copy_state.get_value()
    success = False
    new_row, new_col = move_func(row, col)
    if not_out_of_limits(new_row, new_col) and copy_state.is_player_disk(row, col, value) and copy_state.is_free_space(new_row, new_col):
        success = True
    else:
        new_row = row
        new_col = col
    return success, new_row, new_col

In [9]:
move_functions_list = [up_function,down_function,left_function,right_function,up_left_function,up_right_function,down_left_function,down_right_function]

def get_possible_actions(state, row, column, game=np.zeros(shape=(4,4), dtype=np.int32), debug=False):
    possible_actions = copy.deepcopy(game)
    value = state.get_value()
    for move in move_functions_list:
        success, possible_move_row, possible_move_column = possible_move_function(state, row, column, move)
        if success:
            possible_actions[possible_move_row, possible_move_column] = value
            if debug:
                print(move)
                print(possible_actions)
    return possible_actions

In [10]:
game = get_possible_actions(State("MAX"), 0, 0, np.zeros(shape=(4,4), dtype=np.int32), True)

<function down_function at 0x000001CC029E9D30>
[[ 0  0  0  0]
 [-1  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]
<function right_function at 0x000001CC029E9EE0>
[[ 0 -1  0  0]
 [-1  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]


In [11]:
map_values = {0:"A", 1:"B", 2:"C", 3:"D"}
#map_values = {"A":0, "B":1, "C":2, "D":3}

def actions_2(state):
    possible_actions_matrix = np.zeros(shape=(4,4), dtype=np.int32)
    value = state.get_value()
    pos_values_player = np.argwhere(state.game_board == value)
    pos_values_player = [(pos[0], pos[1]) for pos in pos_values_player]
    output_positions = []
    output_actions = []
    for pos in pos_values_player:
        possible_actions_matrix = get_possible_actions(state, pos[0], pos[1], possible_actions_matrix)
        pos_possible_actions_player = np.argwhere(possible_actions_matrix == value)
        pos_possible_actions_player = [(pos[0], pos[1]) for pos in pos_possible_actions_player]
        print("pos", pos_possible_actions_player)
        pos_possible_actions_player = [map_values[pos[1]] + str(pos[0]+1) for pos in pos_possible_actions_player]
        print("pos 2", pos_possible_actions_player)
        output_positions.append((pos[0], pos[1]))
        output_actions.append(pos_possible_actions_player)
    return output_positions, output_actions, possible_actions_matrix

def actions_3(state):
    value = state.get_value()
    pos_values_player = np.argwhere(state.game_board == value)
    pos_values_player = [(pos[0], pos[1]) for pos in pos_values_player]
    output_positions = []
    output_actions = []
    for pos in pos_values_player:
        possible_actions_matrix = np.zeros(shape=(4,4), dtype=np.int32)
        possible_actions_matrix = get_possible_actions(state, pos[0], pos[1], possible_actions_matrix)
        pos_possible_actions_player = np.argwhere(possible_actions_matrix == value)
        pos_possible_actions_player = [(pos[0], pos[1]) for pos in pos_possible_actions_player]
        pos_possible_actions_player = [map_values[pos[1]] + str(pos[0]+1) for pos in pos_possible_actions_player]
        output_positions.append((pos[0], pos[1]))
        #output_positions.append(map_values[pos[1]] + str(pos[0]+1))
        output_actions.append(pos_possible_actions_player)
    possible_actions_matrix = np.zeros(shape=(4,4), dtype=np.int32)
    #possible_actions_matrix = get_possible_actions(state, pos[0], pos[1], possible_actions_matrix)
    for pos in output_positions:
        possible_actions_matrix = np.logical_or(possible_actions_matrix, get_possible_actions(state, pos[0], pos[1], np.zeros(shape=(4,4), dtype=np.int32)))
        possible_actions_matrix = np.where(possible_actions_matrix, value, 0)
    output_positions = [map_values[pos[1]] + str(pos[0]+1) for pos in output_positions]
    return output_positions, output_actions, possible_actions_matrix

def actions(state):
    value = state.get_value()
    output_positions = []
    output_actions = []
    for pos in np.argwhere(state.game_board == value):
        possible_actions_matrix = get_possible_actions(state, *pos, np.zeros(shape=(4,4), dtype=np.int32))
        pos_possible_actions_player = [map_values[pos[1]] + str(pos[0]+1) for pos in np.argwhere(possible_actions_matrix == value)]
        output_positions.append(pos)
        output_actions.append(pos_possible_actions_player)
    possible_actions_matrix = np.logical_or.reduce([get_possible_actions(state, *pos, np.zeros(shape=(4,4), dtype=np.int32)) for pos in output_positions])
    possible_actions_matrix = np.where(possible_actions_matrix, value, 0)
    output_positions = [map_values[pos[1]] + str(pos[0]+1) for pos in output_positions]
    return output_positions, output_actions, possible_actions_matrix

In [12]:
state = State()
print(f"INITIAL PLAYER: {state.current_player} with value {state.get_value()} has this options: ")
output_positions, possible_actions, matrix = actions(state)
print(output_positions)
print (possible_actions)
print(matrix)
print("'game board'")
print(state.game_board)
print(f"PLAYER: {state.current_player} with value {state.get_value()} moved {output_positions[0]} to {possible_actions[0][1]}")
state = move_function(state, 0, 0, right_function)
print("-------------")
print(f"PLAYER: {state.current_player} with value {state.get_value()} has this options: ")
output_positions, possible_actions, matrix = actions(state)
print(output_positions)
print (possible_actions)
print(matrix)
print("'game board'")
print(state.game_board)
print(f"PLAYER: {state.current_player} with value {state.get_value()} moved {output_positions[0]} to {possible_actions[0][0]}")
state = move_function(state, 3, 0, up_function)
print("-------------")
print(f"PLAYER: {state.current_player} with value {state.get_value()} has this options: ")
output_positions, possible_actions, matrix = actions(state)
print(output_positions)
print (possible_actions)
print(matrix)
print("'game board'")
print(state.game_board)

INITIAL PLAYER: MAX with value -1 has this options: 
['A1', 'B2', 'C3', 'D4']
[['B1', 'A2'], ['B1', 'C1', 'A2', 'A3'], ['D2', 'D3', 'B4', 'C4'], ['D3', 'C4']]
[[ 0 -1 -1  0]
 [-1  0  0 -1]
 [-1  0  0 -1]
 [ 0 -1 -1  0]]
'game board'
[[-1  0  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
PLAYER: MAX with value -1 moved A1 to A2
-------------
PLAYER: MIN with value 1 has this options: 
['D1', 'C2', 'B3', 'A4']
[['C1', 'D2'], ['C1', 'D2', 'D3'], ['A2', 'A3', 'B4', 'C4'], ['A3', 'B4']]
[[0 0 1 0]
 [1 0 0 1]
 [1 0 0 1]
 [0 1 1 0]]
'game board'
[[ 0 -1  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
PLAYER: MIN with value 1 moved D1 to C1
-------------
PLAYER: MAX with value -1 has this options: 
['B1', 'B2', 'C3', 'D4']
[['A1', 'C1', 'A2'], ['A1', 'C1', 'A2'], ['D2', 'D3', 'B4', 'C4'], ['D3', 'C4']]
[[-1  0 -1  0]
 [-1  0  0 -1]
 [ 0  0  0 -1]
 [ 0 -1 -1  0]]
'game board'
[[ 0 -1  0  1]
 [ 0 -1  1  0]
 [ 1  1 -1  0]
 [ 0  0  0 -1]]


In [30]:
dictionary = {"A":0, "B":1, "C":2, "D":3}
def get_result(state, disk, action):
    result_state = copy.deepcopy(state)
    row, col = int(disk[1])-1, dictionary[disk[0]]
    new_row, new_col = int(action[1])-1, dictionary[action[0]]
    move_func = None
    if new_row == row + 1 and new_col == col:
        move_func = down_function
    elif new_row == row - 1 and new_col == col:
        move_func = up_function
    elif new_row == row and new_col == col - 1:
        move_func = left_function
    elif new_row == row and new_col == col + 1:
        move_func = right_function
    elif new_row == row - 1 and new_col == col - 1:
        move_func = up_left_function
    elif new_row == row - 1 and new_col == col + 1:
        move_func = up_right_function
    elif new_row == row + 1 and new_col == col - 1:
        move_func = down_left_function
    elif new_row == row + 1 and new_col == col + 1:
        move_func = down_right_function
    if move_func is None:
        raise ValueError(f"Invalid action: {action}")
    result_state = move_function(result_state, row, col, move_func)
    return result_state

In [31]:
state = State("MAX")
print(f"INITIAL PLAYER: {state.current_player}")
print(state.game_board)
print(f"PLAYER: {state.current_player} moved B2 to B1")
state = get_result(state, 'B2', 'B1')
print(state.game_board)
print(f"PLAYER: {state.current_player} moved C2 to C1")
state = get_result(state, 'C2', 'C1')
print(state.game_board)

INITIAL PLAYER: MAX
[[-1  0  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
PLAYER: MAX moved B2 to B1
[[-1 -1  0  1]
 [ 0  0  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
PLAYER: MIN moved C2 to C1
[[-1 -1  1  1]
 [ 0  0  0  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]


In [32]:
def evaluate_state(state):
    game_board = state.game_board
    max_score = 0
    min_score = 0

    # Check horizontal columns
    for row in range(4):
        if np.all(game_board[row, :] == -1):
            max_score += 1
        elif np.all(game_board[row, :] == 1):
            min_score += 1

    # Check vertical columns
    for col in range(4):
        if np.all(game_board[:, col] == -1):
            max_score += 1
        elif np.all(game_board[:, col] == 1):
            min_score += 1

    # Check square 2x2
    for i in range(3):
        for j in range(3):
            submatrix = game_board[i:i+2, j:j+2]
            if np.all(submatrix == -1):
                max_score += 1
            elif np.all(submatrix == 1):
                min_score += 1

    # Check corner positions
    if game_board[0, 0] == -1:
        max_score += 1
    elif game_board[0, 0] == 1:
        min_score += 1

    if game_board[0, 3] == -1:
        max_score += 1
    elif game_board[0, 3] == 1:
        min_score += 1

    if game_board[3, 0] == -1:
        max_score += 1
    elif game_board[3, 0] == 1:
        min_score += 1

    if game_board[3, 3] == -1:
        max_score += 1
    elif game_board[3, 3] == 1:
        min_score += 1

    return -1 * (max_score - min_score)

In [33]:
max_depth = 3
def cut_off(state, depth):
    return depth == max_depth

def min_max_cutoff(state):
    max_depth = 3
    best_piece = None
    best_action = None
    alpha = -np.inf
    beta = np.inf
    
    state.current_player = "MAX"
    v = -np.inf
    state_positions, state_actions, possible_actions_matrix = actions(state)
    for i in range(len(state_positions)):
        for action in state_actions[i]:
            val = min_value(get_result(copy.deepcopy(state), state_positions[i], action), alpha, beta, 1)
            if val > v:
                v = val
                best_piece = state_positions[i]
                best_action = action
    
    return best_piece, best_action

def min_value(state, alpha, beta, depth):
    state.current_player = "MIN"
    if cut_off(state, depth):
        return evaluate_state(state)
    v = np.inf
    state_positions, state_actions, possible_actions_matrix = actions(state)
    for i in range(len(state_positions)):
        for action in state_actions[i]:
            val = max_value(get_result(copy.deepcopy(state), state_positions[i], action), alpha, beta, depth+1)
            v = min(v, val)
            if v <= alpha:
                return v
            beta = min(beta, v)
    return v

def max_value(state, alpha, beta, depth):
    state.current_player = "MAX"
    if cut_off(state, depth):
        return evaluate_state(state)
    v = -np.inf
    state_positions, state_actions, possible_actions_matrix = actions(state)
    for i in range(len(state_positions)):
        for action in state_actions[i]:
            val = min_value(get_result(copy.deepcopy(state), state_positions[i], action), alpha, beta, depth+1)
            v = max(v, val)
            if v >= beta:
                return v
            alpha = max(alpha, v)
    return v

In [48]:
stateplus = State()

#print(stateplus.game_board)
stateee = State()
print(stateplus.game_board)
stateplus.game_board = np.array([[0, 1, 0, -1],
                               [0, 0, 1, 1],
                               [0, 0, 0, -1],
                               [0, 1, -1, -1]])
player_positions, possible_actions, matrix = actions(stateplus)
print('Player positions:', player_positions)
print('Possible actions:', possible_actions)
print(stateplus.game_board)
min_max_cutoff(stateplus)

[[-1  0  0  1]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 1  0  0 -1]]
Player positions: ['D1', 'D3', 'C4', 'D4']
Possible actions: [['C1'], ['C3'], ['B3', 'C3'], ['C3']]
[[ 0  1  0 -1]
 [ 0  0  1  1]
 [ 0  0  0 -1]
 [ 0  1 -1 -1]]


('D1', 'C1')